In [32]:
import importlib
import functions

importlib.reload(functions)  # This will reload the functions module with any changes made

from functions import *

In [11]:
BASE_LINK = 'https://data.who.int/countries'

In [12]:
driver = start_browser()

In [15]:
open_page(driver, BASE_LINK)

In [33]:
counteries = get_list_of_countries(driver)

In [36]:
print(f"Total {len(counteries)} countries found")

Total 194 countries found


## NOW use only 1st country present in counteries list and write the required functions

In [37]:
country = counteries[0]
open_page(driver,country)

## NAJAM

**Things to do**
- 1 Name of country
- 2 Current health expenditure (% of GDP)
- 3 WHO region
- 4 World Bank income level
- 5 Population growth rate
- 6 Leading cause of death
- 7 Leading cause of under 5 mortality
- 8 extract data from population graph from 2000 to 2024 (get twenty rows w.r.t two columns year and population)


In [ ]:
# get_country_name(driver)

# get_current_health_expenditure(driver)

# get_who_region(driver)

# get_world_bank_income_level(driver)

# get_population_growth_rate(driver)

# get_leading_cause_of_death(driver)

# get_leading_cause_of_death(driver)

# get_leading_cause_of_under_5_mortality(driver)

# get_population_data(driver)

## SHAHZAD

**Things to do**

- 1 extract data from life expentency at birth graph (get twenty data points from 2000 to 2024 with two columns year and total (not male or female))
- 2 extract data from Healthy life expectancy (HALE) at birth graph (get twenty data points from 2000 to 2024 with two columns year and total (not male or female))
- 3 A function to extract **visit indicator** links for following categories
   - 3.1 Number of new HIV infections
   - 3.2 People living with tuberculosis (TB)
   - 3.3 Malaria cases
   - 3.4 Probability of dying from non-communicable diseases
   - 3.5 Road traffic deaths
   - 3.6 Suicide deaths
   - 3.7 Prevalence of hypertension
   - 3.8 Adult obesity
   - 3.9 Tobacco use
   - 3.10 Alcohol consumption
   - 3.11 Safely managed sanitation
   - 3.12 Fine particulate matter
   - 3.13 UHC index score
   - 3.14 Births attended by skilled health personnel
   - 3.15 Family planning
   - 3.16 DTP3 immunization
   - 3.17 MCV2 immunization
   - 3.18 Interventions against NTDs
   - 3.19 Density of doctors
   - 3.20 Density of nurses
   - 3.21 Density of pharmacists
   - 3.22 Density of dentists
   - 3.23 WASH development assistance
   - 3.24 Financial hardship
- 4 A general function to extract visit indicators of this kind of pages (https://data.who.int/indicators/i/49AC786/77D059C?m49=004)
   - 4.1 return 24 rows with two columns year and count.
- 5 a general function to extract the other kind of function
   - 5.1 get 23-24 years data from line graph displayed on indicator page (example https://data.who.int/indicators/i/CCCEBB2/217795A?m49=004)

In [ ]:
# get_life_expentency_data(driver)

# get_health_life_expectancy_data(driver)

# get_all_indicators(driver)



# get_indicator_data_for_group_A(driver)
# group A example https://data.who.int/indicators/i/49AC786/77D059C?m49=004


# get_indicator_data_for_group_ B(driver)
# group B example https://data.who.int/indicators/i/CCCEBB2/217795A?m49=004